In [ ]:
import deepinv
import torch
from torchvision import datasets, transforms

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 48
image_size = 32

transform = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.0,), (1.0,)),
    ]
)


In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform,
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
lr = 1e-4
epochs = 5

model = deepinv.models.DiffUNet(
    in_channels=1, 
    out_channels=1, 
    pretrained=None).to(device) # type: ignore
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
mse = torch.nn.MSELoss()

In [ ]:
beta_start = 1e-4
beta_end = 0.02
timesteps = 1000

In [ ]:
betas = torch.linspace(beta_start, beta_end, timesteps, device=device)
alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)

In [ ]:
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    for data, _ in train_loader:
        imgs = data.to(device)
        noise = torch.randn_like(imgs)
        t = torch.randint(0, timesteps, (imgs.size(0),), device=device)

        noisy_imgs = (
            sqrt_alphas_cumprod[t, None, None, None] * imgs +
            sqrt_one_minus_alphas_cumprod[t, None, None, None] * noise
        )
        optimizer.zero_grad()
        estimated_noise = model(noisy_imgs, t, type_t="timestep")
        loss = mse(estimated_noise, noise)
        loss.backward()
        optimizer.step()

In [ ]:
torch.save(
    model.state_dict(), 
    "trained_diffusion_model.pth"
)